In [46]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from datetime import timedelta
from datetime import datetime
import f_OTN
import plotly.express as px
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
import plotly.graph_objects as go

In [45]:
data = {
    'Participant': ['Peyton CRAIG', 'Corentin LE CLEZIO', 'George MILLS',
                    'Joseph DENG', 'Nico BOKETTA', 'Mason COHEN',
                    'Luke SHAW', 'Jamie WEBB'],
    'Temps': [timedelta(minutes=1, seconds=44, milliseconds=12), timedelta(minutes=1, seconds=44, milliseconds=25), 
              timedelta(minutes=1, seconds=46, milliseconds=11), timedelta(minutes=1, seconds=46, milliseconds=41),
              timedelta(minutes=1, seconds=46, milliseconds=74), timedelta(minutes=1, seconds=46, milliseconds=92),
              timedelta(minutes=1, seconds=48, milliseconds=14), timedelta(minutes=1, seconds=49, milliseconds=39)]
}

df = pd.DataFrame(data)
df['Time'] = df['Temps'].apply(lambda x: x.total_seconds())
fig = px.bar(df, x='Participant', y='Time',
             color='Time',
             labels={'Participant':'Participants',
                     'Time':'Time (in seconds)'}, 
             height=400,
             color_discrete_sequence=px.colors.qualitative.Dark24)

fig.update_layout(
    title='ON track night 800m - Final 4 (Men)',
    xaxis_title='Participants',
    yaxis_title='Temps (en secondes)',
    yaxis_tickformat='s',
    yaxis_type='log',  
    uniformtext_minsize=8, 
    uniformtext_mode='hide' 
)
fig.update_traces(marker_line_color='rgb(8,48,107)', 
                  marker_line_width=1.5)
fig.show()

In [35]:
R = ['Meeting National <br> Est Lyonnais',
     'Meeting International <br> de Montreuil',
     'Meeting International <br> de Marseille',
     'Meeting Stanislas Nancy',
     'Meeting International <br> de Forbach',
     "Meeting International <br> de Troyes Aube",
    'Nationales Abendsportfest - Pfungstadt, TSV Stadion, Pfungstadt', 
    'ON Track Night Vienna <br> PB - minimas olympiques']

T = [timedelta(minutes=1, seconds=45, milliseconds=240),
     timedelta(minutes=1, seconds=46, milliseconds=760),
     timedelta(minutes=1, seconds=45, milliseconds=810),
     timedelta(minutes=1, seconds=45, milliseconds=220),
     'DNF',
     timedelta(minutes=1, seconds=46, milliseconds=940),
     timedelta(minutes=1, seconds=45, milliseconds=00),
     timedelta(minutes=1, seconds=44, milliseconds=250)]

D = [datetime(year=2024, month=5, day=4),
     datetime(year=2024, month=5, day=16),
     datetime(year=2024, month=5, day=22),
     datetime(year=2024, month=5, day=25),
     datetime(year=2024, month=5, day=26),
     datetime(year=2024, month=6, day=16),
     datetime(year=2024, month=6, day=19),
     datetime(year=2024, month=6, day=22)]

corentin = pd.DataFrame(f_OTN.remplir_dico_athle(R,T,D))

corentin['sec'] = corentin['Time'].apply(lambda x: 100 if x == 'DNF' else x.total_seconds())
corentin.sort_values(by='Date', inplace=True)

def format_race_date(row):
    if row['Time'] == 'DNF':
        return f"<b> DNF </b> <br> {row['Races']} ({row['Date'].strftime('%d/%m/%Y')})"
    else:
        return f"{row['Races']} <br> ({row['Date'].strftime('%d/%m/%Y')})"

corentin['Race_Date'] = corentin.apply(format_race_date, axis=1)
fig = px.line(corentin, x='Date', y='sec', color='Races', markers=True,
              range_x=[corentin['Date'].min(), corentin['Date'].max()],
              range_y=[corentin['sec'].min() - 5, corentin['sec'].max() + 5],
              title="Évolution des chronos sur 800m de Corentin LE CLEZIO (été 2024)",
              labels={'Date': 'Date de la course', 'sec': 'Chrono (en secondes)'},
              line_shape='spline',
              hover_name='Race_Date')


fig.update_layout(
    width=1000,  # Largeur du graphique
    height=700,  # Hauteur du graphique
    showlegend=True,
    legend=dict(
        traceorder='normal',
        font=dict(size=10),
        orientation="v",
        x=1,
        y=1
    )
)


frames = []


for i in range(len(corentin)):
    frame_data = corentin.iloc[:i+1]  
    frame_annotations = [
        dict(
            x=corentin.iloc[j]['Date'],
            y=corentin.iloc[j]['sec'],
            text=corentin.iloc[j]['Race_Date'],
            showarrow=True,
            arrowhead=1,
            xanchor='left',
            yanchor='bottom',  
            ax=20,  
            ay=-30
        )
        for j in range(i+1)
    ]
    trace = px.scatter(corentin.iloc[i:i+1], x='Date', y='sec', color='Races').data[0]
    trace.marker.size = 10  
    
    frames.append({
        'data': [fig.data[0], trace],  
        'layout': {'annotations': frame_annotations}
    })
fig.frames = frames
fig.update_layout(
    updatemenus=[{
        "buttons": [{
            "args": [None, {
                "frame": {"duration": 3000, "redraw": True},  
                "fromcurrent": True,
                "transition": {"duration": 500, "easing": "linear"}
            }],
            "label": "Play",
            "method": "animate"
        }],
        "direction": "left",
        "pad": {"r": 10, "t": 87},
        "showactive": False,
        "type": "buttons",
        "x": 0.1,
        "xanchor": "right",
        "y": 0,
        "yanchor": "top"
    }]
)
#fig.add_annotation(x = datetime(year=2024, month=5, day=26), y=100, text='DNF 26/05/2024', showarrow=True, arrowhead=1)
fig.add_layout_image(
    dict(
        source="https://th.bing.com/th?id=OIF.%2b90RqIvf%2fD8wBBpnHltrsg&rs=1&pid=ImgDetMain",  # URL ou chemin local de votre image
        xref="paper", yref="paper",
        x=0, y=0,  
        sizex=0.4, sizey=0.8,  
        xanchor="left", yanchor="bottom" 
    )
)
fig.update_layout(showlegend=False)
fig.show()


Focus sur Corentin Le Clezio, l'athlète qui a réalisé les minimas olympiques sur 800m. À seulement 24 ans, il a récemment exprimé sa satisfaction quant à ses progrès lors des entraînements, attribués à une approche scientifique innovante. Après avoir rejoint le On Athletics Club (OAC), Le Clezio a revu son programme d'entraînement pour le rendre plus rigoureux et contrôlé, notamment à travers l'utilisation de tests lactates.

Sa progression en détails :

In [68]:
dates = list(range(2018,2025))
times = [timedelta(minutes=1, seconds=54, milliseconds=90),
     timedelta(minutes=1, seconds=50, milliseconds=71),
     timedelta(minutes=1, seconds=49, milliseconds=37),
     timedelta(minutes=1, seconds=48, milliseconds=25),
     timedelta(minutes=1, seconds=48, milliseconds=91),
     timedelta(minutes=1, seconds=45, milliseconds=61),
     timedelta(minutes=1, seconds=44, milliseconds=25)] 

seconds = [time.total_seconds() for time in times]

fig = go.Figure()


fig.add_trace(go.Scatter(
    x=dates,
    y=seconds,
    mode='lines+markers',
    marker=dict(size=10), 
    line=dict(shape='spline'),  
    name='Chronos de Corentin Le Clezio'
))

fig.update_layout(
    title='Évolution des chronos de Corentin Le Clezio',
    xaxis_title='Année',
    yaxis_title='Chrono (en secondes)',
    hovermode='x', 
    hoverlabel=dict(bgcolor='white', font_size=12),  
    template='plotly_white')

minima_olympique_date = 2024
minima_olympique_time = timedelta(minutes=1, seconds=44, milliseconds=25).total_seconds()
fig.add_annotation(
    x=minima_olympique_date,
    y=minima_olympique_time,
    text="Minima olympique",
    showarrow=True,
    arrowhead=2,
    arrowsize=1,
    arrowwidth=2,
    arrowcolor='red',
    ax=0,
    ay=-40,
    align='center'
)
fig.add_layout_image(
    dict(
        source="https://th.bing.com/th/id/OIF.wyNxYlrIxqujWvPbcDS6gg?rs=1&pid=ImgDetMain",  
        xref="paper", yref="paper",
        x=1, y=1,  
        sizex=0.5, sizey=0.5,  
        xanchor="right", yanchor="top" 
    )
)
fig.show()


In [69]:
data = {
    'Participant': ['Bendere OBOYA', 'Charlotte PIZZO', 'Claudia HOLLINGSWORTH',
                    'Kimberley FICENEC', 'Jaylah HANCOCK-CAMERON', 'Khadija BENKASSEM',
                    'Georgia-Maria DESPOLLARI', '	Bregje SLOOT', 'Georgie HARTIGAN', 'Cristina Daniela BALAN'],
    'Temps': [timedelta(minutes=1, seconds=58, milliseconds=56), timedelta(minutes=1, seconds=59, milliseconds=99), 
              timedelta(minutes=2, seconds=00, milliseconds=2), timedelta(minutes=2, seconds=00, milliseconds=79),
              timedelta(minutes=2, seconds=2, milliseconds=3), timedelta(minutes=2, seconds=2, milliseconds=71),
              timedelta(minutes=2, seconds=2, milliseconds=78), timedelta(minutes=2, seconds=2, milliseconds=81),
              timedelta(minutes=2, seconds=3, milliseconds=22), timedelta(minutes=2, seconds=6, milliseconds=85)]
}

df = pd.DataFrame(data)
df['Time'] = df['Temps'].apply(lambda x: x.total_seconds())
fig = px.bar(df, x='Participant', y='Time',
             color='Time',
             labels={'Participant':'Participants',
                     'Time':'Time (in seconds)'}, 
             height=400,
             color_discrete_sequence=px.colors.qualitative.Dark24)

fig.update_layout(
    title='ON track night 800m - Final 2 (Women)',
    xaxis_title='Participants',
    yaxis_title='Temps (en secondes)',
    yaxis_tickformat='s',
    yaxis_type='log',  
    uniformtext_minsize=8, 
    uniformtext_mode='hide' 
)
fig.update_traces(marker_line_color='rgb(8,48,107)', 
                  marker_line_width=1.5)
fig.show()